In [ ]:
from datasets import load_dataset
import torch
import pickle

dataset = load_dataset("squad_v2")
print(dataset)

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to('cuda')

# read from file
# filehandler = open('./squad-v2-berttokenized.obj', 'rb') 
# dataset = pickle.load(filehandler)

def tokenize_function(examples):
    return tokenizer(examples['question'], examples['context'], padding = 'max_length')

dataset = dataset.map(tokenize_function, batched=True)
print(dataset)

In [ ]:
dataset['train'][0]['answers']

In [ ]:
def collate_fn(batch):
    # for i, el in enumerate(batch):
    #     print(i, el['input_ids'])
    batch = {
        'input_ids': torch.stack([torch.tensor(el['input_ids']) for el in batch], dim = 0),
        'token_type_ids': torch.stack([torch.tensor(el['token_type_ids']) for el in batch], dim = 0),
        'attention_mask': torch.stack([torch.tensor(el['attention_mask']) for el in batch], dim = 0),
        'labels': torch.stack([torch.tensor(el['a'])], dim = 0)
    }
    # print(batch)
    return batch

train = torch.utils.data.DataLoader(
    dataset['train'],
    batch_size = 4,
    collate_fn = collate_fn,
    )
print(len(train))
val = torch.utils.data.DataLoader(
    dataset['validation'],
    batch_size = 4,
    collate_fn = collate_fn
    )
print(len(val))

for batch in train:
    print('type(batch)', type(batch))
    batch = {
        'input_ids': batch['input_ids'].to('cuda'),
        'token_type_ids': batch['token_type_ids'].to('cuda'),
        'attention_mask': batch['attention_mask'].to('cuda'),
    }
    # input_ids = torch.stack(batch['input_ids'], dim = 1)
    # token_type_ids = torch.stack(batch['token_type_ids'], dim = 1)
    # attention_mask = torch.stack(batch['attention_mask'], dim = 1)

    # batch = {'input_ids': input_ids,
    #          'token_type_ids': token_type_ids,
    #          'attention_mask': attention_mask}.to('cuda')

    with torch.inference_mode():
        outputs = model(**batch)
    print(outputs)
    # Rest of your code

    # break  # Break the loop after the first batch